In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from erutils.command_line_interface import *

In [ ]:
batch = 4
chunk = 8
channel = 32
head_size = 16
B, T, C = batch, chunk, channel

wei = torch.tril(torch.ones(T, T))
x = torch.randn(B, T, C)

In [ ]:
fprint(x)
fprint(wei)
fprint(x.shape)
fprint(wei.shape)

In [ ]:
wei_u = wei / wei.sum(1, keepdim=True)

In [ ]:
wei_u

In [ ]:
out = wei_u @ x

In [ ]:
out[0]

In [ ]:
x = torch.randn(B, T, C)

In [ ]:
x.shape

In [ ]:
q = query(x)
k = key(x)
wei = q @ k.transpose(-2, -1)  # (B,T,C) @ (B,C,T) -> (B,T,T)

In [ ]:
wei[0]

In [ ]:
trill = torch.tril(torch.ones((T, T)))
wei = wei.masked_fill(trill == 0, float('-inf'))
wei = f.softmax(wei, dim=-1)
wei[0]

In [ ]:
v = value(x)

out = wei @ v
out.shape

In [ ]:
out[0]

In [ ]:
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

In [ ]:
x = torch.randn((B, T, C))
q = query(x)
k = key(x)
v = value(x)
wei = q @ k.transpose(-2, -1) * head_size ** -0.5
trill = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(trill == 0, float('-inf'))
wei = f.softmax(wei, dim=-1)
out = wei @ v


In [ ]:
wei[0]

In [ ]:
number_of_embedded = 32
number_of_heads = 16
dp1 = nn.Dropout(0.2)
dp2 = nn.Dropout(0.2)
attn = nn.Linear(number_of_embedded, 3 * number_of_embedded)
proj = nn.Linear(number_of_embedded, number_of_embedded)

In [ ]:
chunk = 64
x = torch.randn(10, chunk, number_of_embedded)
B, T, C = x.shape

In [ ]:
x = attn(x)
print('xShape ::: ', x.shape)
q, k, v = x.split(number_of_embedded, dim=2)

In [ ]:
q.shape

In [ ]:
q.shape


In [ ]:
q = q.view(B, T, number_of_heads, C // number_of_heads).transpose(1, 2)
k = k.view(B, T, number_of_heads, C // number_of_heads).transpose(1, 2)
v = v.view(B, T, number_of_heads, C // number_of_heads).transpose(1, 2)

In [ ]:
v.shape
import math


In [ ]:
att = q @ k.transpose(-2, -1) * (1.0 / math.sqrt(k.size(0)))
att.shape

In [ ]:
bias = torch.tril(torch.ones(T, T).view(1, 1, chunk, chunk))
att = att.masked_fill(bias[:, :, :T, :T] == 0, float('-inf'))
att = F.softmax(att, dim=-1)

In [ ]:
att = dp1(att)

In [ ]:
wei = att @ v

In [ ]:
wei = wei.transpose(1, 2).contiguous().view(B, T, C)

In [ ]:
wei.shape


In [ ]:
import torch

v = torch.randn((5, 100))
v = torch.nn.functional.softmax(v, dim=-1)
print(torch.multinomial(v, 1))